In [26]:
# Begin by importing some necessary modules
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

#Helper function that returns the number of characters different in two strings
def char_diff(str1, str2):
    return sum ( str1[i] != str2[i] for i in range(len(str1)) )

# Messages to be hashed
message_1 = b"Buy 10000 shares of WXYZ stock now!"
message_2 = b"Buy 10000 shares of VXYZ stock now!"

print(f"The two messages differ by { char_diff(message_1, message_2)} characters")

The two messages differ by 1 characters


In [27]:
# Create new SHA-256 hash objects, one for each message
chf_1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
chf_2 = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Update each hash object with the bytes of the corresponding message
chf_1.update(message_1)
chf_2.update(message_2)

# Finalize the hash process and obtain the digests
digest_1 = chf_1.finalize()
digest_2 = chf_2.finalize()

#Convert the resulting hash to hexadecimal strings for convenient printing
digest_1_str = digest_1.hex()
digest_2_str = digest_2.hex()

#Print out the digests as strings 
print(f"digest-1: {digest_1_str}")
print(f"digest-2: {digest_2_str}")

print(f"The two digests differ by { char_diff(digest_1_str, digest_2_str)} characters")

digest-1: 6e0e6261b7131bd80ffdb2a4d42f9d042636350e45e184b92fcbcc9646eaf1e7
digest-2: 6b0abb368c3a1730f935b68105e3f3ae7fd43d7e786d3ed3503dbb45c74ada46
The two digests differ by 57 characters


In [28]:
# import the required crypto functions which will be demonstrated later
from secretpy import Caesar
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

# Set the plaintext we want to encrypt
plaintext=u"this is a strict top secret message for intended recipients only"
print(f"\nGiven plaintext: {plaintext}")


Given plaintext: this is a strict top secret message for intended recipients only


In [29]:
# initialize the required python object for doing Caesar shift encryption
caesar_cipher = Caesar()

# Define the shift, ie the key
caesar_key = 5 
print(f"Caesar shift secret key: {caesar_key}")

# Define the alphabet
alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')
print(f"alphabet: {alphabet}")

Caesar shift secret key: 5
alphabet: ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')


In [30]:
caeser_ciphertext = caesar_cipher.encrypt(plaintext, caesar_key, alphabet)
print(f"Encrypted caeser shift ciphertext: {caeser_ciphertext}")

Encrypted caeser shift ciphertext: ymnxenxefexywnhyeytuexjhwjyerjxxfljektwensyjsijiewjhnunjsyxetsqc


In [31]:
caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: this is a strict top secret message for intended recipients only



In [32]:
# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively
aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: nyelcdgtfiuyxdfm


In [33]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")


AES initialization vector: mrobabyfome yhvk


In [34]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes
sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process
aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b'\xc5\xbf8\xf9\xd4\x04\xfa\x7fxJ\xbe~\xd7\x1f\xff\x89\x08\xf4\xcb\xe4\x81\x9f\xa9\xea\xd1\xa1R\xc6\xd1\xf9\xd3\xa5C\xefNod+[\xe7s\xd9E\xbf\xbd\x13\xd1<\xe0tC\xd4N\x8e\xd2\xab\x950C\x95\xb1\xae\x95\xc2'


In [35]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: this is a strict top secret message for intended recipients only


In [36]:
# Example function to compute the gcd (greatest common divisor) 
def gcd(a,b):
    if b==0:
        return a
    else:
        return gcd(b,a%b)
    
# let's calculate some examples using algorithm
n1=gcd(50,10)
n2=gcd(99,33)
n3=gcd(59,9)

# do the same with the python library call
import math
m1=math.gcd(50,10)
m2=math.gcd(99,33)
m3=math.gcd(59,9)

# Confirm they are the same
assert(n1==m1)
assert(n2==m2)
assert(n3==m3)

# They are - print out the values for explanation
print("gcd(50,10) =",m1)
print("gcd(99,1033) =",m2)
print("gcd(59,9) =",m3)

gcd(50,10) = 10
gcd(99,1033) = 33
gcd(59,9) = 1


In [37]:
# Choosing two prime numbers and keep them secret
p = 13
q = 19
print("The secret prime numbers p and q are:", p, q)

# Calculate n which is the modulus for both the public and private keys
n = p * q
print("modulus n (p*q)=",n)

The secret prime numbers p and q are: 13 19
modulus n (p*q)= 247


In [38]:
# Compute Euler's totient function, φ(n) and keep it secret
phi = (p-1) * (q-1)
print("The secret Euler's function (totient) [phi(n)]:",phi)

# Choose an integer e such that e and φ(n) are coprime
e = 2
while (e < phi):
    if (math.gcd(e, phi)==1):
        break
    else:
        e += 1
print("Public Key (e):",e)

The secret Euler's function (totient) [phi(n)]: 216
Public Key (e): 5


In [39]:
# Compute a value for d such that (d * e) % φ(n) = 1
d = 1
while(True):
    if((d*e) % phi == 1):
        break
    else:
        d += 1
print("Private Key (d):",d)

# Public and Private Key pair
public = (e, n)
private = (d, n)

print(f"The Public key is {public} and Private Key is {private}")

Private Key (d): 173
The Public key is (5, 247) and Private Key is (173, 247)


In [41]:
# Encryption function
def encrypt(plain_text):
    return (plain_text ** e) % n

# Decryption function
def decrypt(cipher_text):
    return (cipher_text ** d) % n

# Simple message to encode
msg = 98

# encrypt then decrypt
enc_msg = encrypt(msg)
dec_msg = decrypt(enc_msg)

print("Original Message:",msg)
print("Encrypted Message:",enc_msg)
print("Decrypted Message:",dec_msg)

Original Message: 98
Encrypted Message: 167
Decrypted Message: 98


In [44]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes

symmetric_key = Fernet.generate_key()
print(f"\nSymmetric key generated by Alice: {symmetric_key}")

# Bob generates a 2048-bit RSA key pair
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()
print(f"Public key broadcast by Bob: {bob_public_key}")
print(f"\nPublic numbers in Bobs' public key: {bob_public_key.public_numbers()}")

# Encryption
ciphertext = bob_public_key.encrypt(
   symmetric_key,
   padding.OAEP(
       mgf=padding.MGF1(algorithm=hashes.SHA256()),
       algorithm=hashes.SHA256(),
       label=None
   )
)

print("Ciphertext:", ciphertext)

# Bob decrypts ciphertext to access the symmetric key
decrypted_symmetric_key = bob_private_key.decrypt(
   ciphertext,
   padding.OAEP(
       mgf=padding.MGF1(algorithm=hashes.SHA256()),
       algorithm=hashes.SHA256(),
       label=None
   )
)

print("Decrypted key:", decrypted_symmetric_key)
assert decrypted_symmetric_key == symmetric_key


Symmetric key generated by Alice: b'uCO-TQF8sh5_feH2_gdeelv1Rl_rfpbP0moOg2wtkV8='
Public key broadcast by Bob: <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey object at 0x103d420d0>

Public numbers in Bobs' public key: <RSAPublicNumbers(e=65537, n=28187510055955778481792063137912264844211376981087607349766998261818864248691619463894860035064635381064091317699303490688875040930610576807722342173689530362529478844930262670984233021341960920126041326144660502903800965943593175558868576024235920356965030118429774467683041328434005345313082449858105852953998320484004712302318626334940094727403217597778976172473768791311953002040029125950250069199013225953788592245822603199031439316571765357733640883731128080704440501577478262808321622804056702341237567460769384895980365818347847069819826381976551587707032699477695994126765921484186693310445101066207115388771)>
Ciphertext: b'\x8fThb^\t6<\xb4\xe4\xad(K\xd1\xd1\x92_\xcfJ\xb0E\xa0\xd3\xbc\xf2}^s<\xe6\xb2\xa1y\xa4(/\xfdP`\xed\xf4<}

In [46]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes, serialization
from os import urandom

# Bob's RSA key pair
private_key_Bob = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key_Bob = private_key_Bob.public_key()

print("Bob's private and public keys created")

Alice_long_secret = urandom(160)  # A 160 byte or 1280 bit random message
print("Alice's secret created")

Alice_encrypted_secret = public_key_Bob.encrypt(
    Alice_long_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print("Alice's secret encrypted")

Bob_decrypted_secret = private_key_Bob.decrypt(
    Alice_encrypted_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

assert Alice_long_secret == Bob_decrypted_secret, "Secrets do not match!"

# if we get here they match
print("Secrets match")

def key_derivation_function(secret, salt):
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,  # Desired key length
        salt=salt,
        info=None,
        backend=None
    )
    return hkdf.derive(secret)

common_salt = urandom(16)  # Random salt accessible to both Alice and Bob

symmetric_key_Alice = key_derivation_function(Alice_long_secret, common_salt)
symmetric_key_Bob = key_derivation_function(Bob_decrypted_secret, common_salt)

assert symmetric_key_Alice == symmetric_key_Bob, "Derived keys do not match!"
print(f"A symmetric key of length {len(symmetric_key_Alice)*8} bits was successfully derived by both Alice and Bob!")

Bob's private and public keys created
Alice's secret created
Alice's secret encrypted
Secrets match
A symmetric key of length 256 bits was successfully derived by both Alice and Bob!


In [47]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

# Generate keys for Bob
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()

# Generate keys for Alice
alice_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
alice_public_key = alice_private_key.public_key()

print("Private and Public keys generated for Bob and Alice.")

# Alice encrypts the message using Bob's public key
ciphertext = bob_public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("ciphertext of symmetric key: ",ciphertext)

# Alice signs the ciphertext using her private key
digest = hashes.Hash(hashes.SHA256())
digest.update(ciphertext)
hash_to_sign = digest.finalize()

signature = alice_private_key.sign(
    hash_to_sign,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    Prehashed(hashes.SHA256())
)

print("signature: ",signature)

# Bob receives the ciphertext and signature
received_ciphertext = ciphertext
received_signature = signature

# Send signature and ciphertext here
print("Sending ciphertext and signature.....")

#Bob creates a hash of the ciphertext using the same algorithm used by Alice
digest = hashes.Hash(hashes.SHA256())
digest.update(received_ciphertext)
hash_to_verify = digest.finalize()

print("hash to verify: ",hash_to_verify)

# Bob verifies the signature using Alice's public key
try:
    alice_public_key.verify(
        received_signature,
        hash_to_verify,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        Prehashed(hashes.SHA256())
    )
    print("The signature is valid.")
except:
    print("The signature is not valid.")

# Bob decrypts the message using his private key
decrypted_message = bob_private_key.decrypt(
    received_ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("Decrypted message:", decrypted_message.decode())

Private and Public keys generated for Bob and Alice.
ciphertext of symmetric key:  b'-\xd9\xb0\xa5\xec\xeb\x03\\3\rmF8x\xbcp\xd2\x92\x91B,9\r\xd2\x00\x8a8\x842\x1d_C\x93\x84\xd5ll&\x8b\xa5\xf7\xf9\x9e\x1e@\x10!\xb92\xd0\x90=\xef`\xa8\x05A\xcdS\x9bW\n\x07}Vz\xad\x92jat\x024\x13\t\xfbl3KA\xe6Js\x9dn|\x1d8F\xdd\x8d\x81\xb3(\xcb{\xc3\xbd\x01.\x8a%\x0b|Q\xd5)3\xda\x12\xeb_\xab#p\x81>i\x88\x0e\xbad\xab5\x84E\xa2G2;\xa5\x13K\xda-\x83\x91Wg\x92g>\xaf\xc0\r\xb27\xb2\xbf\xcf\x9cs}\xd6\xbaH\xaf\x8f\xf4\x1e\xfc\x94\x00\xd7.)\x1b\x0f\x94\xbd\xa5Bef\t\xf8\x16\xc2\xaf\x81\xb4\x1c\xf2O!\xda\x836\xd4jq\x91\xd2\x80\x01\xa7\xe4`\xc8H\x15=l\xedEitI\x80>q\xa1I\xbaF\x8b\nP\x94\xfc\xb1\x11\xa0|\xf1u;\x17\x96L\xd5*y\xe2|2x\x93?b(\xe1\xfd\xd8TZa6\x8eK\x1c\x02\x81j\x9bu'
signature:  b'/y/\xb9\xfa\xd5\xc3\xbd\x0e\xda\x95\xb5\x1dh\xab\xc2\xdd@\x91\xbc\xa4(2[\xdb\x8b\xc5-N^\\\xb6\xcc\x87\xc2W\x8f\xd5/\xb2#\x05\x86(\xc5\x7f\x0f\xa1\xa9\xa9\xca\xe3\x96\x19]\xb1\x8b\x0e\x86xK\x95\xc6\x83\xdd\xe0\x1b\xb6\xecW\xceL\x8a

In [48]:
n = 247  # the modulus
e = 5    # public key number
a = 6    # an integer coprime to n
assert gcd(a, n) == 1
print(f"Checked {n} and {a} are coprime")

r=0
rem = 100
while(rem != 1):
    r += 1
    rem = (a**r) % n
    
print(f'period r is: {r}')
assert a**r % n == 1

print(f"Checked {a}^{r} mod {n} is 1")

# explicitly use as integer
f1 = int ( a**(r/2) - 1)
f2 = int ( a**(r/2) + 1)

print(f"f1 = {f1}")
print(f"f2 = {f2}")

q_found = gcd(f1, n)
print(f'One possible prime factor of n ({n}) is: {q_found}')

# explicit int (to avoid floating point)
p_found = int ( n/q_found )
print(f'The second prime factor of n ({n}) is: {p_found}')

assert n == p_found * q_found

#Compute the totient
phi_found = ( p_found -1 ) * ( q_found - 1 ) 
print(f'The totient is: {phi_found}')

#Recover the private key number d_found by satisfying (d_found * e) % phi_found = 1
d_found = 1
while(True):
    if((d_found*e) % phi_found == 1):
        break
    else:
        d_found += 1
print("Private Key number:",d_found)



Checked 247 and 6 are coprime
period r is: 36
Checked 6^36 mod 247 is 1
f1 = 101559956668415
f2 = 101559956668417
One possible prime factor of n (247) is: 19
The second prime factor of n (247) is: 13
The totient is: 216
Private Key number: 173


In [49]:
#Step 1: Choose a prime `p` and a primitive root `a`
p = 11
a = 7

print(f"prime: {p}")
print(f"primitive root: {a}")

k_A = 4  #Alice's private key
h_A = (a**(k_A)) % p #Alice's public key

print(f"Alice's private key is {k_A} and public key is {h_A}")

k_B = 8  #Bob's private key
h_B = (a**(k_B)) % p #Bob's public key

print(f"Bob's private key is {k_B} and public key is {h_B}")

secret_key_alice  = h_B**k_A % p
secret_key_bob = h_A**k_B % p 
assert secret_key_alice == secret_key_bob
print(f'The shared secret key is: {secret_key_bob}')

prime: 11
primitive root: 7
Alice's private key is 4 and public key is 3
Bob's private key is 8 and public key is 9
The shared secret key is: 5
